In [5]:
###predict where the link will be posted given just the title!
import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import math
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
import math

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

train = pd.read_csv('../code/jan_may_2017_links_cross_posting_info.csv')
all_pages = list(train.buzz_account_display_name.unique())

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
train.columns

Index(['buzz_external_id', 'buzz_post_parent_external_id',
       'buzz_post_created_at', 'buzz_account_display_name',
       'buzz_post_type_name', 'buzz_post_buzz_id', 'buzz_campaign_uri',
       'buzz_campaign_name', 'ext_table1_stats_date', 'ext_external_id',
       ...
       'Tasty One-Pot', 'BuzzFeed Sweaty', 'Tasty Junior',
       'Oh Great, More Politics', 'Einfach Tasty', 'BuzzFeed Steven L.',
       'The Try Guys', 'Ohmygod Yaaa', 'BuzzFeed Chloe', 'Nifty Science'],
      dtype='object', length=113)

In [7]:
starting_features = ['buzz_external_id', 'buzz_account_display_name',
       'buzz_post_type_name', 'buzz_post_buzz_id', 'buzz_campaign_uri',
       'buzz_campaign_name',]
df = train[starting_features]

In [8]:
###for each buzz_id, get the external ID which was posted FIRST!!!
train_grouped = train.groupby(['buzz_post_buzz_id']).agg({'buzz_post_created_at':'min'})

train_grouped = train_grouped.reset_index()

train_grouped = train_grouped.rename(columns={'buzz_post_created_at':'first_post'})

df = pd.merge(train, train_grouped, how='left', on=['buzz_post_buzz_id'])

df = df[df['buzz_post_created_at'] == df['first_post']]

In [9]:
###predict where the link will be posted given just the title!

In [10]:
###predict where the link will be posted given just the title!
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest

pages = []
num_links = []
average_acc = []
average_pre = []
average_rec = []
average_f1 = []

cv_folds = 10
results = pd.DataFrame()
page_clf = []

for test_page in all_pages:
#     test_page = "BuzzFeed"
    X_1 = df[df.buzz_account_display_name == test_page]
    X_0 = df[~(df.buzz_account_display_name == test_page)].sample(n=len(X_1))
    if len(X_1) < 2.5*cv_folds:
        continue
    X_1 = X_1[['buzz_campaign_name', 'buzz_account_display_name']]
    X_0 = X_0[['buzz_campaign_name', 'buzz_account_display_name']]
    X_all = pd.concat([X_1, X_0])

    def get_class(X, page):
        if X == page:
            return 1
        else:
            return 0
    X_all['class'] = X_all['buzz_account_display_name'].apply(get_class, page = test_page)
    X_all = X_all.dropna()
    text_clf = Pipeline([
                         ('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
#                          ('featselect', SelectKBest(k = 100)),
                         ('clf', MultinomialNB()),
#                          ('clf', LinearSVC()),
#                          ('clf', RandomForestClassifier()),
#                          ('clf', LogisticRegression()),
    ])
    fold = 0
    kf = KFold(n_splits = cv_folds, shuffle = True)

    accuracy = []
    precision = []
    recall = []
    fscore = []

    X = X_all['buzz_campaign_name']
    y = X_all['class']

    for train_index, test_index in kf.split(X):
#         print("Fold = ", fold)
        fold+= 1
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        text_clf.fit(X_train, y_train)
        y_pred = text_clf.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        prec, rec, fs, sup = precision_recall_fscore_support(y_test, y_pred)
        accuracy.append(acc)
        precision.append(prec)
        recall.append(rec)
        fscore.append(fs)
    print("Page: ", test_page, "#links: ", len(X_1), "Accuracy: ", np.mean(accuracy), "Precision: ", np.mean(precision), "Recall: ", np.mean(recall), "Fscore:", np.mean(fscore))
    num_links.append(len(X_1))
    average_acc.append(np.mean(accuracy))
    average_pre.append(np.mean(precision))
    average_rec.append(np.mean(recall))
    average_f1.append(np.mean(fscore))
    pages.append(test_page)
    page_clf.append(text_clf)
results['page'] = pages
results['num_links'] = num_links
results['accuracy'] = average_acc
results['precision'] = average_pre
results['recall'] = average_rec
results['fscore'] = average_f1

Page:  BuzzFeed Quiz #links:  1316 Accuracy:  0.875759016016 Precision:  0.898844144799 Recall:  0.875758561026 Fscore: 0.873606419703
Page:  BuzzFeed Japan #links:  966 Accuracy:  0.820404892901 Precision:  0.887943532199 Recall:  0.831010073475 Fscore: 0.798715085114
Page:  BuzzFeed UK #links:  560 Accuracy:  0.683928571429 Precision:  0.769137887508 Recall:  0.685801124998 Fscore: 0.655485054184
Page:  BuzzFeed Brasil #links:  1022 Accuracy:  0.948098995696 Precision:  0.951898128436 Recall:  0.949638947117 Fscore: 0.947957577092
Page:  BuzzFeed Japan News #links:  616 Accuracy:  0.769433516916 Precision:  0.863745075664 Recall:  0.78241129329 Fscore: 0.7336541182
Page:  BuzzFeed Deutschland #links:  581 Accuracy:  0.975913645741 Precision:  0.97691039408 Recall:  0.976253651664 Fscore: 0.975870332137
Page:  BuzzFeed News #links:  649 Accuracy:  0.82282647585 Precision:  0.835122811148 Recall:  0.825185187816 Fscore: 0.821032303328
Page:  BuzzFeed #links:  978 Accuracy:  0.640096807

/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Page:  BuzzFeed Philippines #links:  161 Accuracy:  0.711458333333 Precision:  0.772315763757 Recall:  0.71685665663 Fscore: 0.691592539693
Page:  BuzzFeed Oz News #links:  179 Accuracy:  0.799126984127 Precision:  0.837008178164 Recall:  0.806845481946 Fscore: 0.789376760497
Page:  BuzzFeed Reader #links:  46 Accuracy:  0.75 Precision:  0.774285714286 Recall:  0.759166666667 Fscore: 0.741789321789
Page:  BuzzFeed Pink #links:  25 Accuracy:  0.94 Precision:  0.958333333333 Recall:  0.95 Fscore: 0.938333333333
Page:  BuzzFeed LGBT #links:  114 Accuracy:  0.758695652174 Precision:  0.79562983177 Recall:  0.766225441225 Fscore: 0.75102783084
Page:  BuzzFeed Partner #links:  89 Accuracy:  0.757189542484 Precision:  0.792532051282 Recall:  0.753564213564 Fscore: 0.74402681157
Page:  BuzzFeed Tech #links:  211 Accuracy:  0.810022148394 Precision:  0.83764680098 Recall:  0.818969364829 Fscore: 0.804061083584
Page:  BuzzFeed Science #links:  73 Accuracy:  0.660476190476 Precision:  0.683596403

In [11]:
results

,page,num_links,accuracy,precision,recall,fscore
0,BuzzFeed Quiz,1316,0.875759,0.898844,0.875759,0.873606
1,BuzzFeed Japan,966,0.820405,0.887944,0.831010,0.798715
2,BuzzFeed UK,560,0.683929,0.769138,0.685801,0.655485
3,BuzzFeed Brasil,1022,0.948099,0.951898,0.949639,0.947958
4,BuzzFeed Japan News,616,0.769434,0.863745,0.782411,0.733654
5,BuzzFeed Deutschland,581,0.975914,0.976910,0.976254,0.975870
6,BuzzFeed News,649,0.822826,0.835123,0.825185,0.821032
7,BuzzFeed,978,0.640097,0.720550,0.643142,0.602419
8,BuzzFeed Canada,251,0.641608,0.702197,0.643989,0.611534
9,BuzzFeed Video,121,0.660500,0.720943,0.661675,0.632334


In [12]:
results.mean()

num_links    357.557692
accuracy       0.778595
precision      0.824024
recall         0.782314
fscore         0.763955
dtype: float64

In [13]:
page_clf

[Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
 Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
 Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='c

In [18]:
#####Query by page and title of link - Should a link with this title be posted on this BF page?
q_page = "BuzzFeed Quiz"
q_test = ["Find Out What your spirit animal is?"]  ###list of titles you wanna query

max_prob = 0
max_page = ""

print("Finding best first page for item titled: ", q_test)

for pg in all_pages[:10]:
    try:
        clf = page_clf[pages.index(pg)]
    except:
#         print("Woops, the model for page ", pg, " is still in the works!")
        continue
        
    prob = clf.predict_proba(q_test)[0][1]
    print(pg, prob)
    if prob > max_prob:
        max_prob = prob
        max_page = pg
    
print("Best fit for this link would be: ", max_page)

Finding best first page for item titled:  ['Find Out What your spirit animal is?']
BuzzFeed Quiz 0.857346901535
BuzzFeed Japan 0.0151556639795
BuzzFeed UK 0.603574725765
BuzzFeed Brasil 0.0127997197183
BuzzFeed Japan News 0.0387170315684
BuzzFeed Deutschland 0.0597336895542
BuzzFeed News 0.381261782623
BuzzFeed 0.650399031633
BuzzFeed Canada 0.641803568741
BuzzFeed Video 0.354736716206
Best fit for this link would be:  BuzzFeed Quiz


In [17]:
max_page, max_prob

('BuzzFeed Quiz', 0.85734690153508308)

In [28]:
X_all

,buzz_campaign_name,buzz_account_display_name,class
1474,Caltech Professor Who Harassed Women Was Also ...,BuzzFeed Science,1
1631,The CEO Of A Huge Chicken Company Denied That ...,BuzzFeed Science,1
1670,A NASA Spacecraft Has Found Northern Lights An...,BuzzFeed Science,1
1782,Scientists Tell Navajo Farmers Their Water Is ...,BuzzFeed Science,1
2218,"Why Americans Are So Damn Unhealthy, In 4 Shoc...",BuzzFeed Science,1
3606,This Is Why You Should Always Shower Before Sw...,BuzzFeed Science,1
4187,A Photographer Captured A Jaw-Dropping Weather...,BuzzFeed Science,1
7874,A Shocking 51-49 Senate Vote Just Kept Obama's...,BuzzFeed Science,1
7937,A Scientist Is Leaving Google For A Mental Hea...,BuzzFeed Science,1
8190,Obama Warns Inequality Could Derail Climate Ch...,BuzzFeed Science,1


In [19]:
dr = train[train['buzz_post_buzz_id'] == 4456887].sort_values('buzz_post_created_at')
def get_previous(X):
    prev = -1
    buzzid = train[train['buzz_external_id'] == X]['buzz_post_buzz_id'].values[0]    
    created_at = train[train['buzz_external_id'] == X]['buzz_post_created_at'].values[0]    
    tf = train[(train['buzz_post_buzz_id'] == buzzid)&(train['buzz_post_created_at'] < created_at)]
    try:
        prev = tf[-1:]['buzz_external_id'].values[0]
        return prev
    except:
        return -1

dr['previous_external_id'] = dr['buzz_external_id'].apply(get_previous)

In [20]:
dr[['previous_external_id', 'buzz_external_id', 'buzz_post_created_at', "buzz_post_buzz_id"]]

,previous_external_id,buzz_external_id,buzz_post_created_at,buzz_post_buzz_id
37197,-1,560066014088988_1228152523946997,2017-02-06 12:48:13.398595,4456887
36967,560066014088988_1228152523946997,491452930867938_1550064698340084,2017-02-06 18:26:53.590087,4456887


In [40]:
###parameter tuning using grid search
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

text_clf = Pipeline([
                     ('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
#                      ('clf', LinearSVC()),
#                      ('clf', RandomForestClassifier()),
#                      ('clf', LogisticRegression()),
                    ])
parameters = {'vect__ngram_range': [(1, 1), (1, 2), (1,3)], 
              'vect__max_features': [10,100,1000, 10000],
              'tfidf__use_idf': (True, False),
              'tfidf__norm': ('l1', 'l2', None),
              'clf': [MultinomialNB(), LinearSVC(), RandomForestClassifier(), LogisticRegression(),],
#               'clf__alpha': (1e-2, 1e-3),
             }

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(X_all['buzz_campaign_name'], X_all['class'])

In [41]:
gs_clf.best_params_

{'clf': LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0),
 'tfidf__norm': 'l1',
 'tfidf__use_idf': True,
 'vect__max_features': 1000,
 'vect__ngram_range': (1, 3)}

In [42]:
gs_clf.best_score_

0.76712328767123283

In [3]:
X_all

NameError: name 'X_all' is not defined